In [40]:
import pygame
import numpy as np
import sys
import math

# =======================
# CLASSES
# =======================

class Point:
    def __init__(self, x, y, vx=0.0, vy=0.0):
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.radius = 10
        self.connected_points = []  # Liste des points reliés à ce point

    def update_position(self, dt):
        # Résistance fluide simple
        damping = 0.9  # ou un peu moins pour plus de réalisme
        self.vx *= damping
        self.vy *= damping
        self.x += self.vx * dt
        self.y += self.vy * dt

    def connect(self, other_point):
        self.connected_points.append(other_point)

class Membre:
    def __init__(self, Pa, Pb, length=100.0, stiffness=100.0):
        self.Pa = Pa
        self.Pb = Pb
        Pa.connect(Pb)
        Pb.connect(Pa)
        self.length = length
        self.stiffness = stiffness

    def avoid_stretch(self):
        dx = self.Pb.x - self.Pa.x
        dy = self.Pb.y - self.Pa.y
        dist = np.hypot(dx, dy)
        if dist == 0:
            return
        force_mag = self.stiffness * (dist - self.length)

        fx = force_mag * (dx / dist)
        fy = force_mag * (dy / dist)

        self.Pa.vx += fx / 2
        self.Pa.vy += fy / 2
        self.Pb.vx -= fx / 2
        self.Pb.vy -= fy / 2

class Articulation:
    def __init__(self, Porigin, Pa, Pb, Angle1, Angle2, Angle3, Timing):
        self.Porigin = Porigin
        self.Pa = Pa
        self.Pb = Pb
        self.Angle1 = Angle1
        self.Angle2 = Angle2
        self.Angle3 = Angle3
        self.Timing = Timing

    # Calcul de la variation de volume d'eau dans l'articulation entre T et T+1
    def volume_change(self, dt):
        # Calcul de l'aire à T 
        aire = 0.5 * np.abs(
            self.Pa.x * (self.Pb.y - self.Porigin.y) +
            self.Pb.x * (self.Porigin.y - self.Pa.y) +
            self.Porigin.x * (self.Pa.y - self.Pb.y)
        )
        
        if aire < 1e-6:
            return
        
        # Calcul des nouvelles coordonnées de chaque point au temps T+1 : 
        Pa_x_futur = self.Pa.x + dt * self.Pa.vx
        Pa_y_futur = self.Pa.y + dt * self.Pa.vy
    
        Porigin_x_futur = self.Porigin.x + dt * self.Porigin.vx
        Porigin_y_futur = self.Porigin.y + dt * self.Porigin.vy
    
        Pb_x_futur = self.Pb.x + dt * self.Pb.vx
        Pb_y_futur = self.Pb.y + dt * self.Pb.vy

        aire_futur = 0.5 * np.abs( #Calcul de l'aire d'un triangle à l'aide de 3 points.
            Pa_x_futur * (Porigin_y_futur - Pb_y_futur) +
            Porigin_x_futur * (Pb_y_futur - Pa_y_futur) +                Pb_x_futur * (Pa_y_futur - Porigin_y_futur)
        )

        # Variation de l'aire = estimation de la variation de volume
        volume = aire - aire_futur
        return volume

    def appliquer_force_angulaire(self, compteur, dt):
        # Déterminer l'angle cible selon le compteur
        phase = compteur % self.Timing
        if phase < self.Timing / 3:
            angle_cible = self.Angle1
        elif phase < 2 * self.Timing / 3:
            angle_cible = self.Angle2
        else:
            angle_cible = self.Angle3

        # Vecteurs Porigin→Pa et Porigin→Pb
        vectA = np.array([self.Pa.x - self.Porigin.x, self.Pa.y - self.Porigin.y])
        vectB = np.array([self.Pb.x - self.Porigin.x, self.Pb.y - self.Porigin.y])

        # Calcul du produit scalaire
        produit_scalaire = np.dot(vectA, vectB)

        # Calcul des normes des vecteurs
        normeA = np.linalg.norm(vectA)
        normeB = np.linalg.norm(vectB)

        # Calcul de l'angle entre les vecteurs en radians
        angle_actuel_rad = np.arccos(produit_scalaire / (normeA * normeB))

        # Conversion en degrés, dans l'intervalle [0, 360]
        angle_actuel_deg = np.degrees(angle_actuel_rad) % 360
        
       # Calcul de l'erreur angulaire
        erreur = (angle_cible - angle_actuel_deg)

        # Force tangentielle constante quelque soit l'erreur
        FORCE_ANGULAIRE = 50  # À ajuster selon les tests
        direction = np.sign(erreur)  # +1 ou -1 selon le sens de la correction
        force = direction * FORCE_ANGULAIRE * np.sin(angle_actuel_rad / 2) ** 0.8 #Fonction pour que le bras ait une force max pour des angles de 180°, 
        print(f"Force / angle {force} / {angle_actuel_deg}")
        
        # Tangentes unitaires (perpendiculaires aux vecteurs vers Porigin)
        tangentA = np.array([-vectA[1], vectA[0]], dtype=float)
        tangentB = np.array([vectB[1], -vectB[0]], dtype=float)
        tangentA /= np.linalg.norm(tangentA)
        tangentB /= np.linalg.norm(tangentB)

        # Appliquer les forces
        self.Pa.vx += force * tangentA[0]
        self.Pa.vy += force * tangentA[1]
        self.Pb.vx += force * tangentB[0]
        self.Pb.vy += force * tangentB[1]

        # === 3e loi de Newton : réaction de l'eau sur les bras ===
        # On applique une force opposée à celle qu'on a utilisée pour faire tourner
        

        # === Réaction sur l'origine (3ème et 2ème lois de Newton) ===
        if abs(angle_actuel_rad) > np.radians(70): #Angle ouvert, 3ème loi de Newton car eau supposée immobile
            self.Pa.vx -= force * tangentA[0] 
            self.Pa.vy -= force * tangentA[1]
            self.Pb.vx -= force * tangentB[0]
            self.Pb.vy -= force * tangentB[1]
            self.Porigin.vx -= force * (tangentA[0] + tangentB[0])/2
            self.Porigin.vy -= force * (tangentA[1] + tangentB[1])/2
            
        else : #Angle fermé, hypothèse d'eau immobile entre les bras plus acceptable donc utilisation de la 2nde loi de Newton

            variation_volume = self.volume_change(dt)

            if variation_volume <= 0:
                return

            print(f"Variation volume / distance entre les points : {variation_volume} / {np.hypot(self.Pa.x - self.Pb.x, self.Pa.y - self.Pb.y)}")

            # Calcul de la vitesse d'expulsion
            vitesse = variation_volume / (np.hypot(self.Pa.x - self.Pb.x, self.Pa.y - self.Pb.y) *100* dt) #en unité de distance/s
            print(f"vitesse : {vitesse}")

            # Force = masse × accélération ≈ variation_volume * vitesse / 100*dt
            # ρ fluide = 1, donc masse = aire
            force_prop = 0.005*variation_volume * vitesse / dt # coef d'efficacité
            print(f"Force prop : {force_prop}")
            FORCE_PROPULSION_MAX = 1000 #On ajoute une force max pour éviter les Pbs quand angle proche de 0
            force_prop = min(force_prop, FORCE_PROPULSION_MAX)

            # Direction approximative : perpendiculaire à l’ouverture
            direction = np.array([-(vectA[1] + vectB[1]), vectA[0] + vectB[0]])
            direction = direction / (np.linalg.norm(direction) + 1e-6)

            # Répartir la force sur les 3 points : 
            self.Porigin.vx -= force_prop * direction[1]/3
            self.Porigin.vy -= force_prop * direction[0]/3

            self.Pa.vx -= force_prop * direction[1]/3
            self.Pa.vy -= force_prop * direction[0]/3

            self.Pb.vx -= force_prop * direction[1]/3
            self.Pb.vy -= force_prop * direction[0]/3


class Camera:
    def __init__(self, screen_width, screen_height):
        self.screen_width = screen_width
        self.screen_height = screen_height
        self.offset_x = 0
        self.offset_y = 0

    def center_on(self, points):
        # Calcul du centre de gravité moyen des points
        avg_x = sum(p.x for p in points) / len(points)
        avg_y = sum(p.y for p in points) / len(points)
        
        self.offset_x = self.screen_width // 2 - avg_x
        self.offset_y = self.screen_height // 2 - avg_y

    def apply(self, x, y):
        return int(x + self.offset_x), int(y + self.offset_y)


# =======================
# INITIALISATION
# =======================

pygame.init()
font = pygame.font.SysFont("Arial", 24)
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("Simulation Aquatique")
clock = pygame.time.Clock()

# Couleurs
BLUE = (0, 191, 255)
WHITE = (255, 255, 255)
LIGHT_BLUE = (173, 216, 230)
GRID_COLOR = (200, 230, 255)

# Caméra
camera = None
#camera = Camera(width, height)

# Objets
p1 = Point(300, 300)
p2 = Point(300, 400)
p3 = Point(300, 500)

# Membre
membre1 = Membre(p1, p2)
membre2 = Membre(p3, p2)

#Articulations
Ar1An1_deg= 20
Ar1An2_deg= 20
Ar1An3_deg= 220
Ar1Timing = 400
Ar2An1_deg= 90
Ar2An2_deg= 10
Ar2An3_deg= 160
Ar2Timing = 100
articulation1 = Articulation(p2, p1, p3, Ar1An1_deg, Ar1An2_deg, Ar1An3_deg, Ar1Timing)
#articulation2 = Articulation(p3, p4, p5, Ar2An1_deg, Ar2An2_deg, Ar2An3_deg, Ar2Timing)

# Simulation
dt = 0.01
compteur = 0 #compteur de temps (en centièmes de secondes)
running = True
simulation_active = False

# =======================
# BOUCLE PRINCIPALE
# =======================
while running:
    screen.fill(LIGHT_BLUE)

    # === Événements ===
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_RETURN:
                simulation_active = not simulation_active
            if event.key == pygame.K_c:  # Fermer l'application avec la touche "C"
                running = False

    # === Caméra centrée sur le centre de gravité des points ===
    #camera.center_on([p1, p2, p3])

    # === Grille fixe ===
    spacing = 50
    #grid_offset_x = p1.x - width // 2  # Décalage de la grille par rapport à l'objet
    #grid_offset_y = p1.y - height // 2
    grid_offset_x = 0
    grid_offset_y = 0
    for x in range(0, 3*width + spacing, spacing):
        pygame.draw.line(screen, GRID_COLOR, (x - grid_offset_x, 0), (x - grid_offset_x, height), 1)
    for y in range(0, 3*height + spacing, spacing):
        pygame.draw.line(screen, GRID_COLOR, (0, y - grid_offset_y), (width, y - grid_offset_y), 1)
        

    # === Physique ===
    if simulation_active:
        # Appliquer la force de correction d'étirement
        membre1.avoid_stretch()
        membre2.avoid_stretch()
        articulation1.appliquer_force_angulaire(compteur, dt)

        # Mettre à jour les positions
        p1.update_position(dt)
        p2.update_position(dt)
        p3.update_position(dt)

        # Calcul de la vitesse de Porigin
        vx = p2.vx
        vy = p2.vy
        speed = math.hypot(vx, vy)

        # Affichage de la vitesse
        speed_text = font.render(f"Vitesse : {speed:.2f}px/s", True, (0, 0, 0))
        screen.blit(speed_text, (10, 40))

    # === Dessin des objets ===
    """
    pygame.draw.line(screen, BLUE, camera.apply(p1.x, p1.y), camera.apply(p2.x, p2.y), 10)
    pygame.draw.line(screen, BLUE, camera.apply(p2.x, p2.y), camera.apply(p3.x, p3.y), 10)

    pygame.draw.circle(screen, WHITE, camera.apply(p1.x, p1.y), p1.radius)
    pygame.draw.circle(screen, WHITE, camera.apply(p2.x, p2.y), p2.radius)
    pygame.draw.circle(screen, WHITE, camera.apply(p3.x, p3.y), p3.radius)
    """

    pygame.draw.line(screen, BLUE, (p1.x, p1.y), (p2.x, p2.y), 10)  
    pygame.draw.line(screen, BLUE, (p2.x, p2.y), (p3.x, p3.y), 10)

    pygame.draw.circle(screen, WHITE, (p1.x, p1.y), p1.radius)
    pygame.draw.circle(screen, WHITE, (p2.x, p2.y), p2.radius)
    pygame.draw.circle(screen, WHITE, (p3.x, p3.y), p3.radius)

    # Affichage du timer
    timer_text = font.render(f"Temps : {compteur / 100:.2f}s", True, (0, 0, 0))
    screen.blit(timer_text, (10, 10))

    # === Mise à jour de l'affichage ===
    pygame.display.flip()
    clock.tick(100)
    compteur +=1

# =======================
# FERMETURE
# =======================
pygame.quit()
sys.exit()


Force / angle -50.0 / 180.0
Force / angle -49.99959500574078 / 179.4843414650581
Force / angle -49.99659439985542 / 178.50467466838984
Force / angle -49.98726074200287 / 177.10789333021867
Force / angle -49.96688822425362 / 175.33726377021637
Force / angle -49.930276927572905 / 173.23370209652987
Force / angle -49.87214597700989 / 170.83692847972264
Force / angle -49.787502042439044 / 168.1861392516139
Force / angle -49.67194946709726 / 165.31998274846043
Force / angle -49.52191506333443 / 162.27595530330697
Force / angle -49.33477691347654 / 159.0896471552726
Force / angle -49.108921684997746 / 155.79434510541893
Force / angle -48.84377831505061 / 152.42125083741544
Force / angle -48.53985957971004 / 149.00011679116284
Force / angle -48.19878854734132 / 145.5597133905148
Force / angle -47.82323338187205 / 142.12751143311286
Force / angle -47.4166756671645 / 138.72839666913654
Force / angle -46.98301890466996 / 135.38292747220058
Force / angle -46.5261645692254 / 132.10615279583502
For

KeyboardInterrupt: 